In [1]:
from langchain_core.messages import HumanMessage

In [2]:
groq_api_key=''

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model='Llama3-8b-8192',groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002516C70F610>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002516C6FF940>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [4]:
model.invoke([HumanMessage(content="Hello My name is Pratik and I am working in DH")])
# this info is going to model and we will get the ans

AIMessage(content="Hello Pratik! \n\nIt's nice to meet you. \n\nDH stands for a lot of things - could you tell me what field you work in?  Knowing that will help me understand your context better and maybe even have a more relevant conversation with you. \n\nFor example, are you in:\n\n* **Digital Health?**\n* **Data Handling?**\n* **Distribution Hubs?**\n* **Something else entirely?** \n\n\nLet me know! 😊\n", response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 26, 'total_tokens': 130, 'completion_time': 0.21416199, 'prompt_time': 0.00264524, 'queue_time': None, 'total_time': 0.21680723}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4675c498-03c6-49ac-916a-11bc4ccd5326-0', usage_metadata={'input_tokens': 26, 'output_tokens': 104, 'total_tokens': 130})

In [6]:
from langchain_core.messages import AIMessage
model.invoke([
        HumanMessage(content="Hello My name is Pratik and I am working in DH"),
        AIMessage (content="Hello Pratik! \n\nIt's nice to meet you. \n\nDH stands for a lot of things - could you tell me what field you work in?  Knowing that will help me understand your context better and maybe even have a more relevant conversation with you. \n\nFor example, are you in:\n\n* **Digital Health?**\n* **Data Handling?**\n* **Distribution Hubs?**\n* **Something else entirely?** \n\n\nLet me know! 😊\n"),
        HumanMessage(content="Hey whats my name and where I work")
])
# this is just eg where LLM model is able to remember things or not

AIMessage(content="You said your name is Pratik and you work in DH. \n\nIs there anything else you'd like to tell me about your work in DH? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 146, 'total_tokens': 183, 'completion_time': 0.077000949, 'prompt_time': 0.007630499, 'queue_time': None, 'total_time': 0.084631448}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-aaa1ad50-cd5f-45a0-9220-84d60fdb2e80-0', usage_metadata={'input_tokens': 146, 'output_tokens': 37, 'total_tokens': 183})

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [12]:
#this method will create different session and will remember the message w.r.t session
#when we call this method it will check if session id os present or not if present will return the session id with message history
# this session_id will help to distinguish from 1 session to another seesion
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [13]:
config={'configurable':{'session_id':'chat1'}}

In [18]:
# based on session_id = chat1 it will remember below all message history
response=with_message_history.invoke(
    [HumanMessage(content="Hello My name is Pratik and I am working in DH")],
        config=config
)

In [19]:
response.content

"Hello Pratik! \n\nIt's nice to meet you.  \n\nDH stands for quite a few things. Could you tell me what industry or field DH stands for in your case? \n\nKnowing that will help me understand your work better and perhaps have a more relevant conversation with you. 😊  \n"

In [20]:
# so this will give answer because we have given the same session id where we fed the info to that specific session
with_message_history.invoke(
    [HumanMessage(content="What is my name")],
        config=config
)

AIMessage(content="Your name is Pratik. 😊  \n\nI remember that from our earlier conversation. \n\n\n\nLet me know if there's anything else I can help you with!  \n\n", response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 189, 'total_tokens': 228, 'completion_time': 0.078335573, 'prompt_time': 0.016789078, 'queue_time': None, 'total_time': 0.095124651}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e46ee51-9782-4e22-87c5-45125e21d1ac-0', usage_metadata={'input_tokens': 189, 'output_tokens': 39, 'total_tokens': 228})

In [21]:
config={'configurable':{'session_id':'chat2'}}
# so this will give no answer because we have not  fed the info to that specific session 'session_id':'chat2'
with_message_history.invoke(
    [HumanMessage(content="What is my name")],
        config=config
)

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 18, 'total_tokens': 58, 'completion_time': 0.080836348, 'prompt_time': 0.002321174, 'queue_time': None, 'total_time': 0.083157522}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-48b74172-1893-42e1-9c08-3d4b0b9191f8-0', usage_metadata={'input_tokens': 18, 'output_tokens': 40, 'total_tokens': 58})

In [22]:
# based on session_id = chat1 it will remember below all message history
response=with_message_history.invoke(
    [HumanMessage(content="Hello My name is Abhishek and I work in DH")],
        config=config
)

In [23]:
with_message_history.invoke(
    [HumanMessage(content="What is my name")],
        config=config
)

AIMessage(content='Your name is Abhishek.  \n\nI remember from our previous conversation!  😊 \n\n\n\n', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 136, 'total_tokens': 157, 'completion_time': 0.042, 'prompt_time': 0.008751043, 'queue_time': None, 'total_time': 0.050751043}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f76c96c8-1ab4-49bc-bacb-4964a53667f5-0', usage_metadata={'input_tokens': 136, 'output_tokens': 21, 'total_tokens': 157})

In [24]:
with_message_history.invoke(
    [HumanMessage(content="Where I work")],
        config=config
)

AIMessage(content="You mentioned you work in DH, but didn't specify which company.  \n\nDo you want to tell me?  I'm curious!  😊  \n", response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 167, 'total_tokens': 203, 'completion_time': 0.072753871, 'prompt_time': 0.007919157, 'queue_time': None, 'total_time': 0.080673028}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b5221200-250f-4eee-85df-d8b93d73057a-0', usage_metadata={'input_tokens': 167, 'output_tokens': 36, 'total_tokens': 203})

In [ ]:
# so based on ur chat seesion it will remember the chat history